In [2]:
import pandas as pd
import numpy as np
import joblib

test_a_base = pd.read_csv('data/test_a_base.csv')
test_a_op = pd.read_csv('data/test_a_op.csv')
test_a_trans = pd.read_csv('data/test_a_trans.csv')

In [3]:
ohe_encoder = joblib.load('one_encoder.pkl')

In [4]:
test_a_base_non_nan = test_a_base.drop(columns=['service3_level'],axis=1)

test_a_base_non_nan['sex'].fillna('category 0',inplace=True)
test_a_base_non_nan['provider'].fillna('category 0',inplace=True)
test_a_base_non_nan['province'].fillna('21c43413032d5522',inplace=True)
test_a_base_non_nan['city'].fillna('e3685345c54a1cef',inplace=True)

user = test_a_base_non_nan['user'].values
test_a_base_non_user = test_a_base_non_nan.drop(columns=['user'],axis=1)
category_value = test_a_base_non_user.select_dtypes('O').values
one_hot_value = ohe_encoder.transform(category_value).toarray()

number_value = test_a_base_non_user.select_dtypes('int64').values
df_base = pd.DataFrame()
df_base['user'] = user
base = np.concatenate([one_hot_value,number_value],axis=1)
v_df = pd.DataFrame(base)
df_base = pd.concat([df_base,v_df],axis=1)

In [79]:
df_base.iloc[:,-21:]

,583,584,585,586,587,588,589,590,591,592,...,594,595,596,597,598,599,600,601,602,603
0,0.0,24853.0,24713.0,24719.0,24712.0,24712.0,24719.0,24719.0,24706.0,24755.0,...,24706.0,24706.0,24731.0,25747.0,24969.0,24798.0,24719.0,24804.0,24712.0,24706.0
1,0.0,25011.0,24743.0,24712.0,24712.0,24706.0,24712.0,24706.0,24706.0,24719.0,...,24706.0,24706.0,24725.0,24785.0,24785.0,24719.0,24743.0,24719.0,24706.0,24706.0
2,0.0,24877.0,24744.0,24719.0,24725.0,24712.0,24743.0,24706.0,24712.0,24780.0,...,24706.0,24706.0,24761.0,32221.0,27462.0,24969.0,24767.0,24969.0,24712.0,24706.0
3,0.0,24925.0,24715.0,24712.0,24712.0,24706.0,24737.0,24712.0,24706.0,24737.0,...,24706.0,24706.0,24725.0,27734.0,26827.0,24956.0,24761.0,24925.0,24712.0,24706.0
4,0.0,24877.0,24706.0,24712.0,24712.0,24706.0,24706.0,24706.0,24706.0,24725.0,...,24706.0,24706.0,24725.0,24731.0,24731.0,24712.0,24731.0,24712.0,24712.0,24706.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24310,0.0,24859.0,24741.0,24719.0,24719.0,24706.0,24719.0,24712.0,24706.0,24743.0,...,24706.0,24706.0,24822.0,25959.0,25078.0,24767.0,24737.0,24774.0,24719.0,24712.0
24311,0.0,24889.0,24714.0,24725.0,24725.0,24706.0,24706.0,24706.0,24706.0,24761.0,...,24706.0,24706.0,24712.0,27395.0,26802.0,24895.0,24737.0,25072.0,24712.0,24706.0
24312,0.0,24950.0,24709.0,24719.0,24719.0,24706.0,24712.0,24706.0,24706.0,24755.0,...,24706.0,24706.0,24725.0,25620.0,25096.0,24755.0,24755.0,24755.0,24712.0,24706.0
24313,0.0,24925.0,24740.0,24731.0,24712.0,24731.0,24706.0,24706.0,24712.0,24767.0,...,24706.0,24706.0,24737.0,26160.0,25065.0,24761.0,24749.0,24755.0,24712.0,24706.0


In [75]:
df_base.head()

,user,0,1,2,3,4,5,6,7,8,...,594,595,596,597,598,599,600,601,602,603
0,TestA_21427,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,...,24706.0,24706.0,24731.0,25747.0,24969.0,24798.0,24719.0,24804.0,24712.0,24706.0
1,TestA_08694,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,...,24706.0,24706.0,24725.0,24785.0,24785.0,24719.0,24743.0,24719.0,24706.0,24706.0
2,TestA_13449,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,...,24706.0,24706.0,24761.0,32221.0,27462.0,24969.0,24767.0,24969.0,24712.0,24706.0
3,TestA_10937,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,...,24706.0,24706.0,24725.0,27734.0,26827.0,24956.0,24761.0,24925.0,24712.0,24706.0
4,TestA_14194,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,...,24706.0,24706.0,24725.0,24731.0,24731.0,24712.0,24731.0,24712.0,24712.0,24706.0


In [7]:
op_df = pd.DataFrame()
group = test_a_op.groupby(['user'])['user'].count()
op_df['user']=group.index
op_df['count']=group.values

count_column=test_a_op.columns[1:-1]
for column in count_column:
    op_df[column] = test_a_op.groupby(['user'])[column].apply(lambda x:len(set(x))).values
    
from datetime import timedelta
def timedelta2sec(delta):
    day = delta.split('days')[0].strip()
    h,m,s = delta.split('days')[1].strip().split(':')
    sec=timedelta(days=int(day),hours=int(h),minutes=int(m),seconds=float(s)).total_seconds()
    return sec

test_a_op['time_diff_sec'] = test_a_op['tm_diff'].apply(timedelta2sec)
op_df['time_std']=test_a_op.groupby('user')['time_diff_sec'].agg('std').values

In [8]:
op_df

,user,count,op_type,op_mode,op_device,ip,net_type,channel,ip_3,time_std
0,TestA_00000,27,5,5,2,3,2,2,3,36675.368227
1,TestA_00001,185,8,9,2,8,2,2,7,162816.897975
2,TestA_00002,100,13,13,2,4,3,3,4,392196.728665
3,TestA_00003,63,8,8,2,3,2,2,3,392827.866653
4,TestA_00004,64,11,11,2,5,2,3,5,127194.483892
...,...,...,...,...,...,...,...,...,...,...
20453,TestA_24307,23,2,2,1,2,2,2,2,177092.696598
20454,TestA_24309,68,8,8,3,12,3,3,10,400430.625806
20455,TestA_24311,187,8,8,2,13,4,2,13,349679.145323
20456,TestA_24312,46,7,9,3,6,2,2,6,332094.681771


In [15]:
trans_df = pd.DataFrame()
group = test_a_trans.groupby(['user'])['user'].count()
trans_df['user']=group.index
trans_df['count']=group.values

count_column=list(test_a_trans.columns[1:4])+list(test_a_trans.columns[5:9])
for column in count_column:
    trans_df[column] = test_a_trans.groupby(['user'])[column].apply(lambda x:len(set(x))).values
    
for operate in ['max','min','mean','median','std']:
    trans_df['amount_'+operate]=test_a_trans.groupby(['user'])['amount'].agg(operate).values

In [16]:
df = pd.merge(df_base, op_df, on='user', how='left')

In [17]:
df = pd.merge(df, trans_df, on='user', how='left')

In [67]:
model = joblib.load('model.pkl')

In [68]:
x = df.drop('user',axis=1)
res = model.predict_proba(x)[:,1]

In [69]:
np.mean(res)

0.2936408510710686

In [70]:
df_res = pd.DataFrame()
df_res['user'] = test_a_base['user']
df_res['prob'] = res

In [71]:
df_res.to_csv('result.csv', index=False)

In [72]:
len(test_a_base)

24315